<a href="https://colab.research.google.com/github/YiL17/102-Flowers-Classification/blob/master/Causal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  # set default figure size
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import minimize_scalar

def maximize(g, a, b, args):
    """
    Maximize the function g over the interval [a, b].
    We use the fact that the maximizer of g on any interval is
    also the minimizer of -g.  The tuple args collects any extra
    arguments to g.
    Returns the maximal value and the maximizer.
    """
    objective = lambda x: -g(x, *args)
    result = minimize_scalar(objective, bounds=(a, b), method='bounded')
    maximizer, maximum = result.x, -result.fun
    return maximizer, maximum

class OptimalGrowthModel:
    def init(self,
                 u,            # utility function
                 f,            # production function.
                 β=0.96,       # discount factor
                 α=0.3,
                 μ=0,          # shock location parameter
                 σ=0.1,        # shock scale parameter
                 grid_max=20,
                 grid_size=120,
                 shock_size=250,
                 seed=1234):
        self.u, self.f, self.β, self.α, self.μ, self.σ = u, f, β, α, μ, σ
        # Set up grid
        self.grid = np.linspace(1e-4, grid_max, grid_size)
        # Store shocks (with a seed, so results are reproducible)
        np.random.seed(seed)
        self.shocks = np.exp(np.random.normal(μ, σ, shock_size))

    def state_action_value(self, c, y, v_array):
        """
        Right hand side of the Bellman equation.
        """
        u, f, β, shocks = self.u, self.f, self.β, self.shocks
        v = interp1d(self.grid, v_array, fill_value="extrapolate")
        return u(c) + β * np.mean(v(f(y - c) * shocks))

def T(v, og):
    """
    The Bellman operator.  Updates the guess of the value function
    and also computes a v-greedy policy.
      * og is an instance of OptimalGrowthModel
      * v is an array representing a guess of the value function
    """
    v_new = np.empty_like(v)
    v_greedy = np.empty_like(v)
    for i in range(len(grid)):
        y = grid[i]
        # Maximize RHS of Bellman equation at state y
        c_star, v_max = maximize(og.state_action_value, 1e-10, y, (y, v))
        v_new[i] = v_max
        v_greedy[i] = c_star
    return v_greedy, v_new

def v_star(y, α, β, μ):
    """
    True value function
    """
    c1 = np.log(1 - α * β) / (1 - β)
    c2 = (μ + α * np.log(α * β)) / (1 - α)
    c3 = 1 / (1 - β)
    c4 = 1 / (1 - α * β)
    return c1 + c2 * (c3 - c4) + c4 * np.log(y)

def σ_star(y, α, β):
    """
    True optimal policy
    """
    return (1 - α * β) * y

α = 0.3
def fcd(k):
    return k**α

og = OptimalGrowthModel(u=np.log, f=fcd)
grid = og.grid

v_init = v_star(grid, α, og.β, og.μ)    # Start at the solution

def solve_model(og, v_init, max_iter):
    """
    Solve model by iterating with the Bellman operator for a given number of iterations.

    """
    v = v_init
    for _ in range(max_iter):
        v_greedy, v = T(v, og)
    return v

# Solve the model for different numbers of iterations
iterations = [1, 10, 100, 1000, 10000]
results = [solve_model(og, 5 * np.log(grid), n) for n in iterations]

# Plot the results
fig, ax = plt.subplots()
colors = plt.cm.viridis(np.linspace(0, 1, len(iterations)))
for idx, (n, v) in enumerate(zip(iterations, results)):
    ax.plot(grid, v, color=colors[idx], lw=2, alpha=0.6, label=f'N = {n}')

# Plot the true value function
ax.plot(grid, v_star(grid, α, og.β, og.μ), 'k--', lw=2, alpha=0.8, label='True value function')

ax.legend()
ax.set(ylim=(-40, 10), xlim=(np.min(grid), np.max(grid)))
plt.show()

# Calculate the distance between the actual solution and the numerical one for N = 10000
v_numerical = results[-1]
v_analytical = v_star(grid, α, og.β, og.μ)
distance = np.max(np.abs(v_numerical - v_analytical))
print(f'Distance between the actual solution and the numerical one after 10000 iterations: {distance:.6f}')

TypeError: OptimalGrowthModel() takes no arguments

In [ ]:
from causalml.dataset import synthetic_data

ModuleNotFoundError: No module named 'causalml'

In [ ]:
# Import the sample AB data
import pandas as pd

file_url = "https://msalicedatapublic.z5.web.core.windows.net/datasets/ROI/multi_attribution_sample.csv"
ab_data = pd.read_csv(file_url)
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KDD2021_CausalML/case_1_data_causalml.csv.gz')
# df = pd.read_csv('/content/sample_data/')
print(ab_data.shape)

(2000, 11)


In [ ]:
ab_data.head()

,Global Flag,Major Flag,SMC Flag,Commercial Flag,IT Spend,Employee Count,PC Count,Size,Tech Support,Discount,Revenue
0,1,0,1,0,45537,26,26,152205,0,1,17688.3630
1,0,0,1,1,20842,107,70,159038,0,1,14981.4356
2,0,0,0,1,82171,10,7,264935,1,1,32917.1389
3,0,0,0,0,30288,40,39,77522,1,1,14773.7686
4,0,0,1,0,25930,37,43,91446,1,1,17098.6982


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
df['target'].value_counts()

NameError: name 'df' is not defined

# Define Features

In [ ]:
OUTCOME_COL = 'y_col'
TREATMENT_COL = 'target'
SCORE_COL = 'pihat'
GROUPBY_COL = 'billing_zip'
NAN_INT = -98765    # A random integer to impute missing values with

In [ ]:
PROPENSITY_FEATURES = [
 'prob_features_0',
 'prob_features_1',
 'prob_features_2',
 'prob_features_3',
 'prob_features_4',
 'prob_features_5',
 'prob_features_6',
 'prob_features_7',
 'prob_features_8',
 'prob_features_9',
 'prob_features_10',
 'prob_features_11',
 'prob_features_12',
 'prob_features_13',
 'prob_features_14',
 'prob_features_15',
 'prob_features_16',
 'prob_features_17',
 'prob_features_18',
 'prob_features_19',
 'prob_features_20',
 'prob_features_21',
 'prob_features_22',
 'prob_features_23',
 'prob_features_24',
 'prob_features_25',
 'prob_features_26',
 'prob_features_27']

MATCHING_COVARIATES = [
 'prob_features_11',
 'prob_features_17',
 'prob_features_18',
 'prob_features_19',
 'prob_features_20',
 'prob_features_21',
 'prob_features_22',
 'prob_features_23',
 'prob_features_24'] + [SCORE_COL]

INFERENCE_FEATURES = PROPENSITY_FEATURES + [SCORE_COL]

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import beta

# Random explore fixed period
def ad_simulator_random_explore(horizon, k=None, alpha=None, beta=None, explore_rate=None, p_s=None):
    if p_s is not None:
        k = len(p_s)
    else:
        p_s = np.random.beta(alpha, beta, k)
    visits = np.zeros(k, dtype=int)
    clicks = np.zeros(k, dtype=int)

    explore_period = int(horizon * explore_rate)
    for i in range(explore_period):
        machine = np.random.choice(k)
        visits[machine] += 1
        clicks[machine] += np.random.rand() < p_s[machine]

    p_hats = np.zeros(k)
    p_hats[visits > 0] = clicks[visits > 0] / visits[visits > 0]
    learnt_rank = np.argsort(-p_hats)
    best_estimated = learnt_rank[0]
    visits[best_estimated] += horizon - explore_period
    clicks[best_estimated] += np.sum(np.random.rand(horizon - explore_period) < p_s[best_estimated])
    return pd.DataFrame({'p_s': p_s, 'p_hats': p_hats, 'visits': visits, 'clicks': clicks})

# Deterministic explore fixed period
def ad_simulator_deterministic_explore(horizon, k=None, alpha=None, beta=None, explore_rate=None, p_s=None):
    if p_s is not None:
        k = len(p_s)
    else:
        p_s = np.random.beta(alpha, beta, k)
    visits = np.zeros(k, dtype=int)
    clicks = np.zeros(k, dtype=int)

    explore_period = int(horizon * explore_rate)
    for i in range(explore_period):
        machine = i % k
        visits[machine] += 1
        clicks[machine] += np.random.rand() < p_s[machine]

    p_hats = np.zeros(k)
    p_hats[visits > 0] = clicks[visits > 0] / visits[visits > 0]
    learnt_rank = np.argsort(-p_hats)
    best_estimated = learnt_rank[0]
    visits[best_estimated] += horizon - explore_period
    clicks[best_estimated] += np.sum(np.random.rand(horizon - explore_period) < p_s[best_estimated])
    return pd.DataFrame({'p_s': p_s, 'p_hats': p_hats, 'visits': visits, 'clicks': clicks})

# Initial values
horizon = 10**3
k = 100
alpha = 0.1
beta = 9.9
explore_rate = 0.2
n_test = 1000
gain = pd.DataFrame({'random': np.zeros(n_test), 'deterministic': np.zeros(n_test)})

for i in range(n_test):
    p_s = np.random.beta(0.05, 5, k)

    result = ad_simulator_random_explore(horizon=horizon, p_s=p_s, explore_rate=explore_rate)
    gain.at[i, 'random'] = result['p_s'].max() - result['p_s'][result['p_hats'].idxmax()]

    result = ad_simulator_deterministic_explore(horizon=horizon, p_s=p_s, explore_rate=explore_rate)
    gain.at[i, 'deterministic'] = result['p_s'].max() - result['p_s'][result['p_hats'].idxmax()]

# Plotting results
fig = go.Figure()
fig.add_trace(go.Scatter(x=gain.index, y=gain['random'], mode='lines+markers', name='Random'))
fig.add_trace(go.Scatter(x=gain.index, y=gain['deterministic'], mode='lines+markers', name='Deterministic'))
fig.show()

# UCB 1
def ucb(horizon, k=None, alpha=None, beta=None, p_s=None):
    if p_s is not None:
        k = len(p_s)
    else:
        p_s = np.random.beta(alpha, beta, k)
    visits = np.zeros(k, dtype=int)
    clicks = np.zeros(k, dtype=int)
    p_hats = np.zeros(k)
    upper_p = np.ones(k)
    for i in range(horizon):
        arm = np.argmax(upper_p)
        clicks[arm] += np.random.rand() < p_s[arm]
        visits[arm] += 1
        p_hats[arm] = clicks[arm] / visits[arm]
        upper_p[arm] = min(1, p_hats[arm] + 3 / np.sqrt(visits[arm]))
    return pd.DataFrame({'p_s': p_s, 'p_hats': p_hats, 'upper_p': upper_p, 'visits': visits, 'clicks': clicks})

# Run UCB simulation
horizon = 10**3
k = 100
alpha = 0.1
beta = 9.9
p_s = np.random.beta(alpha, beta, k)
result = ucb(horizon, p_s=p_s)
result = result.sort_values(by='p_s', ascending=False)
print(result.head())


         p_s    p_hats   upper_p  visits  clicks
20  0.308599  0.000000  0.948683      10       0
29  0.134199  0.090909  0.995443      11       1
46  0.131218  0.000000  0.948683      10       0
62  0.077581  0.000000  0.948683      10       0
7   0.058233  0.000000  0.948683      10       0


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import beta

# Parameters
a_0 = 0.1
b_0 = 9.9
k = 100
horizon = int(1e3)
lambda_1 = 0e4
lambda_2 = 1e5
ratio_12 = lambda_1 / (lambda_1 + lambda_2)

np.random.seed(42)
p_s = np.random.beta(a_0, b_0, k)
q_s = np.random.beta(a_0, b_0, k)
mu = 1
sigma = 1
max_s = 100 + np.zeros(k)
c_s = np.minimum(np.round(np.exp(np.random.normal(mu, sigma, k)), 1), max_s)
d_s = np.minimum(np.round(np.exp(np.random.normal(mu, sigma, k)), 1), max_s)
c_s0 = c_s.copy()
d_s0 = d_s.copy()
dq = pd.DataFrame({'merchant': np.arange(1, k+1), 'd_s0': d_s0, 'q_s': q_s, 'max_s': max_s})

visits = pd.DataFrame({
    'site': np.zeros(horizon, dtype=int),
    'merchant': np.zeros(horizon, dtype=int),
    'estimated_p': np.zeros(horizon),
    'cost': np.zeros(horizon),
    'is_clicked': np.zeros(horizon, dtype=bool),
    'revenue': np.zeros(horizon)
})

impressions_1 = np.zeros(k, dtype=int)
clicks_1 = np.zeros(k, dtype=int)
impressions_2 = np.zeros(k, dtype=int)
clicks_2 = np.zeros(k, dtype=int)
p_hat = np.zeros(k)
q_hat = np.zeros(k)

for i in range(horizon):
    if i % 100 == 0:
        p_hat = np.random.beta(a_0 + clicks_1, b_0 + impressions_1 - clicks_1)
        q_hat = np.random.beta(a_0 + clicks_2, b_0 + impressions_2 - clicks_2)

    if i % 1000 == 0:
        print(i)

    next_site = 1 if np.random.rand() < ratio_12 else 2

    if next_site == 1:
        ad = np.argmax(c_s * p_hat)
        impressions_1[ad] += 1
        click = np.random.rand() < p_s[ad]
        clicks_1[ad] += click
        visits.iloc[i] = [next_site, ad, p_hat[ad], c_s[ad], click, c_s[ad] if click else 0]
        if click:
            max_s[ad] -= c_s[ad]
            c_s[ad] = min(c_s[ad], max_s[ad])
            d_s[ad] = min(d_s[ad], max_s[ad])
        p_hat[ad] = np.random.beta(a_0 + clicks_1[ad], b_0 + impressions_1[ad] - clicks_1[ad])
    else:
        ad = np.argmax(d_s * q_hat)
        impressions_2[ad] += 1
        click = np.random.rand() < q_s[ad]
        clicks_2[ad] += click
        visits.iloc[i] = [next_site, ad, q_hat[ad], d_s[ad], click, d_s[ad] if click else 0]
        if click:
            max_s[ad] -= d_s[ad]
            c_s[ad] = min(c_s[ad], max_s[ad])
            d_s[ad] = min(d_s[ad], max_s[ad])
        q_hat[ad] = np.random.beta(a_0 + clicks_2[ad], b_0 + impressions_2[ad] - clicks_2[ad])

# Summary statistics
print(visits['site'].value_counts())
print(visits['is_clicked'].value_counts())
print(visits['revenue'].sum())
print(np.sort(max_s)[::-1])
print(visits[visits['site'] == 2]['merchant'].value_counts().sort_values())
print(visits[(visits['site'] == 2) & (visits['is_clicked'])]['merchant'].value_counts().sort_values())

# Merge and plot
to_compare = visits[visits['site'] == 2].merge(dq, on='merchant')
l = 71
print(max_s[l])
print(q_s[l])
plot_data = to_compare[to_compare['merchant'] == l]
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(plot_data)), y=plot_data['estimated_p'] - plot_data['q_s'], mode='lines'))
fig.show()

# Theoretical best
dq['dq'] = dq['d_s0'] * dq['q_s']
dq = dq.sort_values(by='dq', ascending=False)
dq['expected_time'] = (dq['max_s'] / dq['d_s0']) / dq['q_s']

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(visits)), y=visits['revenue'].cumsum(), mode='lines'))
cumsum_expected_time = dq[dq['expected_time'].cumsum() < horizon]
fig.add_trace(go.Scatter(x=cumsum_expected_time['expected_time'].cumsum(), y=cumsum_expected_time['max_s'].cumsum(), mode='lines'))
fig.show()


0
site
2    1000
Name: count, dtype: int64
is_clicked
False    900
True     100
Name: count, dtype: int64
308.90000000000015
[100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.
 100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.
 100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.
 100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.
 100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.
 100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.
 100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.
 100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.  100.
  91.1   0.    0.    0. ]
merchant
31      1
10      1
64      1
65      1
37      1
     ... 
81      8
87     35
6      83
20    261
35    391
Name: count, Length: 83, dtype: int64
merchant
87     1
20    16
6     20
35    63
Name: count, dtype: int64
100.0
0.015156762856848967


# Data PreProcessing

## Categorical Features

In [ ]:
ENCODING_COLS = [
 'cat_features_0',
 'cat_features_1',
 'cat_features_2',
 'cat_features_3',
 'prob_features_15',
 'prob_features_16']

In [ ]:
# Label Encoding
threshold = int(0.001 * df.shape[0])
lbe = LabelEncoder(min_obs=threshold)
df[ENCODING_COLS] = lbe.fit_transform(df[ENCODING_COLS])

NameError: name 'df' is not defined

## Numerical Features

We pre-scaled the features already, so don't need to do any feature transfer here but we do recommd to check the distribution of the nuemerical features and do the transformation accordingly.

## Missing Values

In [ ]:
# Check if there any missing values
relevant_cols = np.union1d(PROPENSITY_FEATURES, np.union1d(MATCHING_COVARIATES, INFERENCE_FEATURES))
df[[c for c in relevant_cols if c in df]].isnull().sum()

[link text](https://)# Build Propensity Model

In [ ]:
params = {
          'objective': 'binary',
          'metric': 'auc',
          'learning_rate': 0.1,
          'num_leaves': 6,
          'bagging_freq': 1,
          'seed': 8,
          'verbose': 0,
          'num_threads': -1,
         }

PROPENSITY_FEATURES = [col for col in set(PROPENSITY_FEATURES + ENCODING_COLS) if col in df.columns]

# Supposons que PROPENSITY_FEATURES, ENCODING_COLS et df sont des variables définies ailleurs dans votre code.

# PROPENSITY_FEATURES est une liste de caractéristiques (features) utilisées dans le contexte d'une analyse de propension.

# ENCODING_COLS est une autre liste de caractéristiques.

# df est un DataFrame pandas qui semble être la source des données.

# L'objectif est de créer une nouvelle liste PROPENSITY_FEATURES qui combine les éléments uniques de PROPENSITY_FEATURES et ENCODING_COLS, en vérifiant si chaque élément est présent dans les colonnes du DataFrame df.

# Explication ligne par ligne :

# 1. [col for col in set(PROPENSITY_FEATURES + ENCODING_COLS)]: C'est une compréhension de liste (list comprehension) qui crée une liste à partir des éléments de PROPENSITY_FEATURES et ENCODING_COLS, en les combinant dans un ensemble (set) pour éliminer les doublons.

# 2. if col in df.columns: La compréhension de liste ne garde que les éléments qui sont présents dans les colonnes du DataFrame df.

# En résumé, cette ligne de code met à jour la liste PROPENSITY_FEATURES en ajoutant les éléments uniques de ENCODING_COLS (éliminant les doublons) tout en s'assurant que seuls les éléments présents dans les colonnes de df sont inclus.


In [ ]:
X_trn, X_val, y_trn, y_val = train_test_split(df[PROPENSITY_FEATURES], df[TREATMENT_COL].values, test_size=0.2)

lgb_train = lgb.Dataset(X_trn, label=y_trn)
lgb_valid = lgb.Dataset(X_val, label=y_val)

clf = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                early_stopping_rounds=20,
                valid_sets=lgb_valid,
                verbose_eval=0)
n_best = clf.best_iteration
print('n_best', n_best)

In [ ]:
p_val = clf.predict(X_val)
print('Validation AUC={:.4f}'.format(roc_auc_score(y_val, p_val)))

In [ ]:
p_train = clf.predict(X_trn)
print('Train AUC={:.4f}'.format(roc_auc_score(y_trn, p_train)))

In [ ]:
df['pihat']  = clf.predict(df[PROPENSITY_FEATURES])

In [ ]:
imp = pd.DataFrame({'feature': PROPENSITY_FEATURES,
                    'gain': clf.feature_importance(importance_type='gain')})
imp.loc[:, 'gain'] = imp.gain / imp.gain.sum()
imp = imp.sort_values('gain', ascending=False).set_index('feature')
imp = imp.reset_index()
imp.head(n=10)

In [ ]:
plot_evaluation(y_val, p_val)

In [ ]:
plot_evaluation(y_trn, p_train)

# Propensity Score Matching

this provides commparable treatment and control groups

In [ ]:
# plot propensity score
rcParams['figure.figsize'] = 8,8
ax = sns.distplot(df[df[TREATMENT_COL] == 0]['pihat'])
sns.distplot(df[df[TREATMENT_COL] == 1]['pihat'])
ax.set_xlim(0, 1)
ax.set_xlabel("propensity scores")
ax.set_ylabel('density')
ax.legend(['Control', 'Treatment'])

In [ ]:
# before matching
create_table_one(df, treatment_col='target', features=MATCHING_COVARIATES)

In [ ]:
matcher = NearestNeighborMatch(caliper=0.01, replace=True)
df_matched = matcher.match(data=df, treatment_col='target', score_cols=['pihat'])

In [ ]:
# after matching

create_table_one(df_matched, treatment_col='target', features=MATCHING_COVARIATES)

In [ ]:
# plot propensity score
ax = sns.distplot(df_matched[df_matched[TREATMENT_COL] == 0]['pihat'])
sns.distplot((df_matched[df_matched[TREATMENT_COL] == 1]['pihat']))
ax.set_xlim(0, 1)
ax.set_xlabel("propensity scores")
ax.set_ylabel('density')
ax.legend(['Control', 'Treatment'])

[testo del link](https://)### Re-Calibrate propensity (`pihat_recal`) after matching

In [ ]:
p_model = ElasticNetPropensityModel()
ohe = OneHotEncoder(min_obs=df.shape[0] * 0.01)
X_cat = ohe.fit_transform(df_matched[ENCODING_COLS]).todense()
X = np.hstack([df_matched[PROPENSITY_FEATURES].values, X_cat])

df_matched['pihat_recal'] = p_model.fit_predict(X, df_matched['target'])

In [ ]:
# plot propensity score
ax = sns.distplot(df_matched[df_matched[TREATMENT_COL] == 0]['pihat_recal'])
sns.distplot((df_matched[df_matched[TREATMENT_COL] == 1]['pihat_recal']))
ax.set_xlim(0, 1)
ax.set_xlabel("propensity scores")
ax.set_ylabel('density')
ax.legend(['Control', 'Treatment'])

# Inference Analysis

In [ ]:
def prepare_inputs(df, p_col='pihat', w_col='target', y_col='post_sum_gross_bookings', inference_features=INFERENCE_FEATURES):
    X =df[INFERENCE_FEATURES].values
    p = df[p_col].values
    w = df[w_col].values
    df[y_col] = df[y_col].fillna(0)
    y = df[y_col].values

    return X, p ,w, y

In [ ]:
INFERENCE_FEATURES_LR =[col for col in INFERENCE_FEATURES if col not in ENCODING_COLS ]

In [ ]:
def run_learners(df, p_col='pihat', w_col='target', y_col='post_sum_gross_bookings', k=5):
    preds_dict = {}

    for base_learner,label_l in zip([BaseSRegressor, BaseTRegressor, BaseXRegressor, BaseRRegressor],['S', 'T','X','R']):
        for model,label_m in zip([LinearRegression, XGBRegressor],['LR', 'XGB']):
            if label_m !='LR':
                X, p, w, y = prepare_inputs(df, p_col, w_col, y_col)
            else:
                ohe = OneHotEncoder(min_obs=df.shape[0] * 0.01)
                X_cat = ohe.fit_transform(df[ENCODING_COLS]).todense()
                X = np.hstack([df[INFERENCE_FEATURES_LR].values, X_cat])
                p = df[p_col].values
                w = df[w_col].values
                df[y_col] = df[y_col].fillna(0)
                y = df[y_col].values

            if label_l == 'S':
                learner = base_learner(learner=model())
                ate, lb, ub = learner.estimate_ate(X=X,treatment=w,y=y,return_ci=True)
                preds_dict['{} Learner ({})'.format(label_l, label_m)] = np.ravel([ate, lb, ub])
            elif label_l == 'T':
                learner = base_learner(learner=model())
                ate, lb, ub = learner.estimate_ate(X=X,treatment=w,y=y)
                preds_dict['{} Learner ({})'.format(label_l, label_m)] = np.ravel([ate, lb, ub])
            elif label_l == 'X':
                learner = base_learner(learner=model())
                ate, lb, ub = learner.estimate_ate(X=X,p=p,treatment=w,y=y)
                preds_dict['{} Learner ({})'.format(label_l, label_m)] = np.ravel([ate, lb, ub])
            elif label_l == 'R':
                learner = base_learner(model(), n_fold=k)
                ate, lb, ub = learner.estimate_ate(X=X,p=p,treatment=w,y=y)
                preds_dict['{} Learner ({})'.format(label_l, label_m)] = np.ravel([ate, lb, ub])
    return preds_dict

In [ ]:
def get_ate_summary(df, preds_dict, w_col='target', y_col='y_col'):
    preds_df = pd.DataFrame(preds_dict).T
    preds_df.columns = ['ATE', 'Lower', 'Upper']
    post_treatment = df[df[w_col]==1][y_col].mean()
    preds_df['Baseline'] = post_treatment - preds_df['ATE']
    preds_df['Lift'] = preds_df['ATE'] / preds_df['Baseline']
    preds_df = preds_df.round(4)
    return preds_df.T

In [ ]:
# After Re-Calibrate
preds = run_learners(df_matched, y_col='y_col', p_col='pihat_recal')
get_ate_summary(df_matched, preds, y_col='y_col')



```
# 此内容为代码格式
```

# Sensitivity Analysis

All Five methods (with One Sided confounding function and default alpha)

In [ ]:
# Calling the Base XLearner class and return the sensitivity analysis summary report
learner_x = BaseXRegressor(LinearRegression())
sens_x = Sensitivity(df=df_matched, inference_features=INFERENCE_FEATURES, p_col='pihat_recal',
                     treatment_col=TREATMENT_COL, outcome_col=OUTCOME_COL, learner=learner_x)
# Here for Selection Bias method will use default one-sided confounding function and alpha (quantile range of outcome values) input
sens_sumary_x = sens_x.sensitivity_analysis(methods=['Placebo Treatment',
                                                     'Random Cause',
                                                     'Subset Data',
                                                     'Random Replace',
                                                     'Selection Bias'], sample_size=0.9)

In [ ]:
# From the following results, refutation methods show our model is pretty robust;
# When alpah > 0, the treated group always has higher mean potential outcomes than the control; when  < 0, the control group is better off.
sens_sumary_x

In [ ]:
import pandas as pd
df = pd.DataFrame({"A":[1,1,3,2,2,1,3], "B":[1,2,3,1,2,3, 1]})

In [ ]:
df

In [ ]:
df

In [ ]:
df["B"].shift(-1)

In [ ]:
df["A"].quantile(95)